## Проектная работа по теме "Линейная алгебра"

Нам необходимо защитить данные клиентов крупной страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Содержание.  <a class="anchor" id="0-bullet">
* [1. Загрузка данных](#1-bullet)
* [2. Умножение матриц](#2-bullet)
* [3. Алгоритм преобразования](#3-bullet)
* [4. Проверка алгоритма](#4-bullet)

## 1. Загрузка данных<a class="anchor" id="1-bullet"></a>
👈[назад к оглавлению](#0-bullet)

<span style="color:purple">Вызовем необходимые библиотеки, откроем файл данных и изучим обшую информацию.

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from IPython.display import display

# считаем файл данных
data  = pd.read_csv('/datasets/insurance.csv')

#  выведем информацию для загруженной таблицы
print(data.info())
display(pd.concat([data.head(2), data.tail(3)]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


<span style="color:purple">В таблице содержится информация о клиентах страховой компании. Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи, а также количество страховых выплат клиенту за последние 5 лет. Последний признак - целевой.

<span style="color:purple">Разобьём данные на признаки и целевое значение и проконтролируем размерность.

In [2]:
features = data.drop(columns='Страховые выплаты')
target = data['Страховые выплаты']

In [3]:
display(features.shape, target.shape)

(5000, 4)

(5000,)

## 2. Умножение матриц<a class="anchor" id="2-bullet"></a>
👈[назад к оглавлению](#0-bullet)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** <span style="color:purple">(б) Качество линейной регрессии не изменится.

**Обоснование:** 
<span style="color:purple">Пусть $P$ - произвольная обратимая матрица. 

<span style="color:purple">$$X'=XP$$ 

<span style="color:purple">$$w'=((X')^TX')^{-1}(X')^Ty$$

<span style="color:purple">$$a'=X'w'=XP((XP)^TXP)^{-1}(XP)^Ty=XP(P^TX^TXP)^{-1}P^TX^Ty=XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty=XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty$$

<span style="color:purple">Как известно, произведение матрицы на обратную даёт единичную матрицу. Поэтому:

<span style="color:purple">$$a'=XE(X^TX)^{-1}EX^Ty=X(X^TX)^{-1}X^Ty=Xw=a$$

<span style="color:purple">Получили, что $a'=a$, т.е. предсказание не изменится.

## 3. Алгоритм преобразования<a class="anchor" id="3-bullet"></a>
👈[назад к оглавлению](#0-bullet)

<span style="color:purple">Создадим случайную матрицу P.

In [4]:
random_matrix = np.random.randn(features.shape[1], features.shape[1])
display(random_matrix)

array([[ 0.87609576, -1.71202669, -0.49090885, -0.64514356],
       [-0.4737338 ,  1.69219054, -0.75721178, -2.36943734],
       [-1.00828105,  0.34930457, -0.52772542, -0.45864587],
       [ 0.65133088,  1.14731232,  0.91847821,  0.04959445]])

<span style="color:purple">Убедимся, что она обратимая. Умножим её на обратную матрицу, при благоприятном раскладе (если обратная существует) должна получиться единичная матрица.

In [5]:
display(np.round(random_matrix.dot(np.linalg.inv(random_matrix))))

array([[ 1., -0.,  0.,  0.],
       [-0.,  1., -0.,  0.],
       [-0., -0.,  1.,  0.],
       [-0.,  0., -0.,  1.]])

<span style="color:purple">Напишем функцию, которая "зашифрует" наши данные.

In [6]:
def new_feat(features):
    new_features = features.dot(random_matrix)
    return new_features

<span style="color:purple">Посмотрим, что получилось.

In [7]:
new_features = new_feat(features)
display(new_features)

,0,1,2,3
0,-50028.635547,17394.321877,-26205.799193,-22846.577443
1,-38335.820177,13352.561820,-20087.479413,-17537.487438
2,-21187.640249,7384.469541,-11104.193066,-9700.276873
3,-42053.965372,14603.831285,-22020.214713,-19175.191615
4,-26328.523755,9162.518642,-13795.326431,-12037.646496
...,...,...,...,...
4995,-36007.595231,12519.849186,-18859.162645,-16439.902479
4996,-52849.382435,18362.241372,-27677.638991,-24113.554663
4997,-34188.899478,11877.563432,-17903.199186,-15595.384421
4998,-32978.382264,11461.217612,-17271.015530,-15050.343806


<span style="color:purple">Проверим, что при "дешифровке" всё возвращается на круги своя.

In [8]:
rec_features = np.round(new_features.dot(np.linalg.inv(random_matrix)))

display(pd.concat([rec_features.head(2), rec_features.tail(3)]))
display(pd.concat([features.head(2), features.tail(3)]))

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,-0.0,46.0,38000.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0
4999,1.0,28.0,40600.0,1.0


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3
4999,1,28.0,40600.0,1


<span style="color:purple">Как видим, всё восстановилось корректно.

## 4. Проверка алгоритма<a class="anchor" id="4-bullet"></a>
👈[назад к оглавлению](#0-bullet)

<span style="color:purple">Напишем свою линейную регрессию и проверим, что предсказания для исходных и "зашифрованных" данных не отличаются.

In [9]:
class LR_Custom:
    def fit(self, features, target):
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return features.dot(self.w) + self.w0   

In [10]:
def liner(features, target, our_model):
    model = our_model
    model.fit(features, target)
    predictions = model.predict(features)
    r2 = r2_score(target, predictions)
    return r2

In [11]:
display(liner(features, target, LR_Custom()))
display(liner(new_features, target, LR_Custom()))

0.4249455028666801

0.42494550286666366

<span style="color:purple">Метрики совпадают.

<span style="color:purple">Убедимся теперь, что качество линейной регрессии из sklearn также не отличается для исходных и "зашифрованных" данных (также применим метрику R2).

In [12]:
display(liner(features, target, LinearRegression()))
display(liner(new_features, target, LinearRegression()))

0.42494550286668

0.4249455028666802

<span style="color:purple">И здесь та же картина. Резюмируя, метрики R2 нашей модели для исходных и "зашифрованных" данных равны между собой и равны метрикам R2 для библиотечной модели из sklearn. Модель написана корректно, "шифрование" работает верно.